In [1]:
# General imports
from pathlib import Path
from dataclasses import dataclass
from typing import Optional, Iterable, Any
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.rcParams["figure.figsize"] = (18, 10)
plt.rcParams["figure.facecolor"] = "white"

# ML imports
import torch
from torch.utils import data
from torch import nn
import torch.nn.functional as F
import einops
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import wandb
import sklearn.manifold

# power_perceiver imports
from power_perceiver.dataset import NowcastingDataset
from power_perceiver.consts import BatchKey
from power_perceiver.data_loader import HRVSatellite, PV, Sun
from power_perceiver.xr_batch_processor import SelectPVSystemsNearCenterOfImage, ReduceNumPVSystems, ReduceNumTimesteps
from power_perceiver.np_batch_processor import EncodeSpaceTime, Topography
from power_perceiver.transforms.satellite import PatchSatellite
from power_perceiver.transforms.pv import PVPowerRollingWindow

/home/jack/miniconda3/envs/power_perceiver/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#DATA_PATH = Path("~/dev/ocf/power_perceiver/data_for_testing/").expanduser()

DATA_PATH = Path(
    "/mnt/storage_ssd_4tb/data/ocf/solar_pv_nowcasting/nowcasting_dataset_pipeline/prepared_ML_training_data/v15/")
assert DATA_PATH.exists()

In [3]:
def get_dataloader(data_path: Path, tag: str) -> data.DataLoader:
    assert tag in ["train", "validation"]
    assert data_path.exists()
    
    xr_batch_processors = [
        SelectPVSystemsNearCenterOfImage(),
        ReduceNumPVSystems(requested_num_pv_systems=8),
        ]
    
    if tag == "train":
        xr_batch_processors.append(ReduceNumTimesteps(requested_timesteps=4))
    
    dataset = NowcastingDataset(
        data_path=data_path,
        data_loaders=[
            HRVSatellite(
                transforms=[PatchSatellite()]
                ), 
            PV(
                transforms=[PVPowerRollingWindow()]
                ),
            Sun(),
        ],
        xr_batch_processors=xr_batch_processors,
        np_batch_processors=[
            EncodeSpaceTime(),
            Topography("/home/jack/europe_dem_2km_osgb.tif"),
            ]
    )

    dataloader = data.DataLoader(
        dataset,
        batch_size=None,
        num_workers=16,
        pin_memory=True,
    )
    
    return dataloader

#train_dataloader = get_dataloader(DATA_PATH)
train_dataloader = get_dataloader(DATA_PATH / "train", tag="train")
val_dataloader = get_dataloader(DATA_PATH / "test", tag="validation")

/home/jack/miniconda3/envs/power_perceiver/lib/python3.10/site-packages/pyresample/image.py:59: FutureWarning: Usage of ImageContainer is deprecated, please use NumpyResamplerBilinear class instead
  warnings.warn(
/home/jack/miniconda3/envs/power_perceiver/lib/python3.10/site-packages/pyproj/crs/crs.py:1256: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  return self._crs.to_proj4(version=version)
/home/jack/miniconda3/envs/power_perceiver/lib/python3.10/site-packages/pyproj/crs/crs.py:1256: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  return self._crs.to_proj4(version=version)
/home/jack/miniconda3/envs/power_perceiver/lib/python3.10/site-

In [4]:
for batch in train_dataloader:
    break

In [5]:
batch[BatchKey.pv].shape

torch.Size([31, 4, 8])

In [6]:
batch[BatchKey.pv_time_utc].shape

torch.Size([31, 4])

In [7]:
batch[BatchKey.pv].shape

torch.Size([31, 4, 8])

In [8]:
from power_perceiver.pytorch_modules.satellite_processor import HRVSatelliteProcessor
from power_perceiver.pytorch_modules.query_generator import QueryGenerator
from power_perceiver.pytorch_modules.self_attention import PerceiverIO


@dataclass(eq=False)  # See https://discuss.pytorch.org/t/typeerror-unhashable-type-for-my-torch-nn-module/109424/6
class Model(pl.LightningModule):
    encoder_query_dim: int = 64
    num_encoder_query_elements: int = 64
    decoder_query_dim: int = 36  # decoder_query will be automatically padded with zeros to get to this size.
    num_fourier_features: int = 16 # TOTAL for both x and y
    pv_system_id_embedding_dim: int = 16
    byte_array_dim: int = 35
    num_encoder_heads: int = 8
    num_decoder_heads: int = 6
    dropout: float = 0.0
    share_weights_across_latent_transformer_layers: bool = False
    num_latent_transformer_encoders: int = 4
    
    # Other params:
    num_elements_query_padding: int = 0  # Probably keep this at zero while using MultiLayerTransformerEncoder or Perceiver IO

    def __post_init__(self):
        super().__init__()
        self.hrvsatellite_processor = HRVSatelliteProcessor()
        
        self.encoder_query = nn.Parameter(
            torch.randn(self.num_encoder_query_elements, self.encoder_query_dim) / 5)
        
        self.decoder_query_generator = QueryGenerator(
            num_fourier_features=self.num_fourier_features,  # TOTAL (for both x and y)
            pv_system_id_embedding_dim=self.pv_system_id_embedding_dim,
            num_elements_query_padding=self.num_elements_query_padding)
        
        self.perceiver_io = PerceiverIO(
            encoder_query_dim=self.encoder_query_dim,
            decoder_query_dim=self.decoder_query_dim,
            byte_array_dim=self.byte_array_dim,
            num_encoder_heads=self.num_encoder_heads,
            num_decoder_heads=self.num_decoder_heads,
            dropout=self.dropout,
            share_weights_across_latent_transformer_layers=self.share_weights_across_latent_transformer_layers,
            num_latent_transformer_encoders=self.num_latent_transformer_encoders,
            )

        self.output_module = nn.Sequential(
            nn.Linear(in_features=self.decoder_query_dim, out_features=self.decoder_query_dim),
            nn.ReLU(),
            nn.Linear(in_features=self.decoder_query_dim, out_features=1),
        )

        # Do this at the end of __post_init__ to capture model topology to wandb:
        self.save_hyperparameters()
        
    def forward(self, x: dict[BatchKey, torch.Tensor]) -> torch.Tensor:       
        original_batch_size = x[BatchKey.pv].shape[0]
        byte_array = self.hrvsatellite_processor(x)
        
        # Get decoder query
        decoder_query = self.decoder_query_generator(x)
        # Pad with zeros if necessary to get up to self.decoder_query_dim:
        decoder_query = self._maybe_pad_with_zeros(decoder_query)            
        
        # Repeat encoder query for each example in the batch:
        encoder_query = einops.repeat(
            self.encoder_query, 
            "element feature -> example element feature",
            example=byte_array.shape[0],
            )
        
        # Run through the Perceiver IO:
        out = self.perceiver_io(
            encoder_query=encoder_query,
            byte_array=byte_array,
            decoder_query=decoder_query,
            )
        
        out = self.output_module(out)
        
        # Reshape back to (batch_size, n_timesteps, ...)
        return einops.rearrange(
            out, 
            "(batch_size n_timesteps) ... -> batch_size n_timesteps ...", 
            batch_size=original_batch_size)
        
    def _maybe_pad_with_zeros(self, tensor: torch.Tensor) -> torch.Tensor:
        num_zeros_to_pad = self.decoder_query_dim - tensor.shape[-1]        
        assert num_zeros_to_pad >= 0, f"{self.query_dim=}, {tensor.shape=}"
        if num_zeros_to_pad > 0:
            zero_padding_shape = tensor.shape[:2] + (num_zeros_to_pad,)
            zero_padding = torch.zeros(*zero_padding_shape, dtype=tensor.dtype, device=tensor.device)
            tensor = torch.concat((tensor, zero_padding), dim=2)
        return tensor
    
    def _training_or_validation_step(
            self, 
            batch: dict[BatchKey, torch.Tensor], 
            batch_idx: int, 
            tag: str
        ) -> dict[str, object]:
        """
        Args:
            batch: The training or validation batch.  A dictionary.
            tag: Either "train" or "validation"
            batch_idx: The index of the batch.
        """
        actual_pv_power = batch[BatchKey.pv]
        #actual_pv_power = torch.nan_to_num(actual_pv_power, nan=0.0)
        actual_pv_power = torch.where(
            batch[BatchKey.pv_mask].unsqueeze(1), 
            actual_pv_power, 
            torch.tensor(0.0, dtype=actual_pv_power.dtype, device=actual_pv_power.device))

        predicted_pv_power = self(batch).squeeze()
        #mse_loss = F.mse_loss(predicted_pv_power, actual_pv_power, reduction="none").mean(dim=1).float()
        #mse_loss = masked_mean(mse_loss, mask=batch[BatchKey.pv_mask])        
        mse_loss = F.mse_loss(predicted_pv_power, actual_pv_power)
        
        self.log(f"{tag}/mse", mse_loss)
        
        return {
            'loss': mse_loss,
            'predicted_pv_power': predicted_pv_power,
            }
    
    def training_step(self, batch: dict[BatchKey, torch.Tensor], batch_idx: int) -> dict[str, object]:
        return self._training_or_validation_step(batch=batch, batch_idx=batch_idx, tag="train")
    
    def validation_step(self, batch: dict[BatchKey, torch.Tensor], batch_idx: int) -> dict[str, object]:
        return self._training_or_validation_step(batch=batch, batch_idx=batch_idx, tag="validation")
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-4)
        return optimizer

In [9]:
model = Model()
#model = Model.load_from_checkpoint(
#    "~/dev/ocf/power_perceiver/notebooks/2022-04-04_train_ML_model/model.ckpt")


In [10]:
model_output = model(batch)

In [11]:
wandb_logger = WandbLogger(
    project="power_perceiver", 
    entity="openclimatefix",
    log_model="all",
    )

# log gradients, parameter histogram and model topology
wandb_logger.watch(model, log="all")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: openclimatefix (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.14 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


In [12]:
from power_perceiver.analysis.plot_timeseries import LogTimeseriesPlots
from power_perceiver.analysis.plot_tsne import LogTSNEPlot

trainer = pl.Trainer(
    gpus=[3],
    max_epochs=-1,
    logger=wandb_logger,
    callbacks=[
        LogTimeseriesPlots(),
        LogTSNEPlot(),
        ]
    )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [13]:
trainer.fit(
    model=model, 
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
    )

/home/jack/miniconda3/envs/power_perceiver/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:342: LightningDeprecationWarning: Base `Callback.on_train_batch_end` hook signature has changed in v1.5. The `dataloader_idx` argument will be removed in v1.7.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5]

  | Name                    | Type                  | Params
------------------------------------------------------------------
0 | hrvsatellite_processor  | HRVSatelliteProcessor | 0     
1 | decoder_query_generator | QueryGenerator        | 32.0 K
2 | perceiver_io            | PerceiverIO           | 1.1 M 
3 | output_module           | Sequential            | 1.4 K 
------------------------------------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.729     Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|                                                             | 0/2 [00:01<?, ?it/s]

/home/jack/miniconda3/envs/power_perceiver/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:982: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


Epoch 0:   0%|                                                                               | 0/8800 [00:00<?, ?it/s]

/home/jack/miniconda3/envs/power_perceiver/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:982: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


...

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f99e7608280>
Traceback (most recent call last):
  File "/home/jack/miniconda3/envs/power_perceiver/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/home/jack/miniconda3/envs/power_perceiver/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
  File "/home/jack/miniconda3/envs/power_perceiver/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f99e7608280>
Traceback (most recent call last):
  File "/home/jack/miniconda3/envs/power_perceiver/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/home/jac

Epoch 19:  95%|██████████████████████████████████████▏ | 8400/8800 [1:18:03<03:43,  1.79it/s, loss=0.0116, v_num=jcp6]

/home/jack/miniconda3/envs/power_perceiver/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:982: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


Epoch 19:  99%|███████████████████████████████████████▍| 8678/8800 [1:18:17<01:06,  1.85it/s, loss=0.0116, v_num=jcp6]

/home/jack/miniconda3/envs/power_perceiver/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:727: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
Exception in thread Thread-55 (_pin_memory_loop):
Traceback (most recent call last):
  File "/home/jack/miniconda3/envs/power_perceiver/lib/python3.10/threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "/home/jack/miniconda3/envs/power_perceiver/lib/python3.10/threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jack/miniconda3/envs/power_perceiver/lib/python3.10/site-packages/torch/utils/data/_utils/pin_memory.py", line 28, in _pin_memory_loop
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/home/jack/miniconda3/envs/power_perceiver/lib/python3.10/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
  File "/home/jack/minic